## Machine Learning Competition 

## Group name : Maverick
## Student Names : 


- Aishwarya Inamdar        : 
19084404    (100%contribution)
- Fasahat Khan             : 
19121474    (100%contribution)
- Kumar Ankitesh           : 
19059426    (100%contribution)
- Mayank Rikh              : 
19018797    (100%contribution)
- Parna Salian             : 
19111002    (100%contribution)

### Summary:
We have implemented a variety of machine learning models to classify whether a software has a bug or not in classification dataset. The highest test accuracy is 99%, achieved by Random Forest with fine-tuned parameters. Feature selections were used to reduce model complexity and overfitting, and improved test accuracies were observed. Because the dataset consists of 22 labels and that the test accuracy of XGBoost model is 98%, we have built successful models for identifying bugs in a software. As Random Forest classifier comprises of decision trees on the randomly selected data samples, it gets the prediction from each tree and selects the best solution by means of voting. The method used in Random Forest is based on divide-and-conquer approach of decision trees generated on randomly split dataset. Finally, it is simpler and more powerful as compared to other non-linear classification algorithms. To further improve the accuracy, we need more data to train our model. Looking forward, there are several extensions to this work that could be done as stated below:
- Dig deeper into multinomialNB to find way to solve its overfitting problem; 
- Gathering more data to balance data distribution; 
- Model stacking: further combining classifiers to improve performance. Also Bagging can be done to decrease the model variance - Real application: input new data for software and transform them into features the same way as we mentioned and apply our machine learning models to predict bugs.
 
### Libraries Used: pandas, numpy and sklearn
In the algorithms used, we are passing the train data set (X) along with its output (y) and the test data set to test the accuracy of the model. We have defined a range of values from -2 to 3 and creating a set of values in variable ‘r’ using Pow() function which are used as parameters (params) stored in Param library. 

### User defined functions: 

- generateCSV(): 
Generates the final result in CSV format containing the corresponding unique IDs of different features of the     testDataset and respective probability.
- calculateAccuracy(): 
The RepeatedKFold() function splits the data set in 3 parts and the cross-validator is repeated 5 times. cross_val_score() gives the score which determines the accuracy based on number of folds ‘kf’
- mle(): 
to find the maximum likelihood estimator.
- bestParamCalculator():  
to evaluate the predictions on the data set based on negated mean squared error.

## Implementation

### Step 1: Importing the requirements and ignoring warnings

In [9]:
import pandas as pd
import numpy as np
import warnings

from sklearn.svm import SVC
import xgboost as xgb
from xgboost import plot_importance
from sklearn.svm import SVC
from pandas import DataFrame

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix 
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.neural_network import MLPClassifier

from sklearn.feature_selection import SelectFromModel

from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.exceptions import ConvergenceWarning

from numpy import sort
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore", category = ConvergenceWarning)
warnings.filterwarnings("ignore", category = FutureWarning)

#dictionary containing various scores
resultsDict = {}

### Step 2: Utility method definitions

- Method - generateCSV()
- Params - probability , testDataset
- Description - This function generates the csv file based on testDataset and probability using data frames.


In [10]:
#auxilary methods
def generateCSV(probability, testDataset):
    print()
    print('************ CSV FILE ***************')
    print()

    idValues = testDataset['Id']
    finalDataset = {
        'Id' : idValues,
        'Category' : probability
    }

    df = DataFrame(finalDataset, columns = ['Id', 'Category'])
    df.to_csv (r'export_dataframe.csv', index = None, header = True)

###### Maximum Likelihood Estimator
Maximum Likelihood estimation selects the parameter values that makes the observed data most probable. The below mle function takes in the output of train dataset as the parameter and calculates the maximum likelihood estimator.

In [11]:
def mle(y):
    print()
    print('************** MAXIMUM LIKELIHOOD ***************')
    print()
    maximumLikelihoodEstimate = y.value_counts(normalize = 1)
    print('Maximum likelihood ', maximumLikelihoodEstimate)

###### bestParamCalculator()
bestParamCalculator method takes in the train data, output of train dataset, model, parameters, boolean values for data pre-processing algorithms and with the help of RandomizedSearchCV function, gets the best model of all the possible permutations of the passed parameters.

In [12]:
def bestParamCalculator(X, y, model, params, applyScalar = False, applyNormalizer = False):

    kf = RepeatedKFold(n_splits = 3, n_repeats = 5, random_state = None) 
    random = RandomizedSearchCV(estimator = model, param_distributions = params, cv = kf, verbose = 2, random_state = 42, n_jobs = -1, scoring='neg_mean_squared_error')
    if applyScalar:
        scaler = RobustScaler().fit(X)
        X = scaler.transform(X)   

    if applyNormalizer:
        normalizer = Normalizer().fit(X)
        X = normalizer.transform(X)

    random.fit(X, y)
    return random.best_estimator_

###### Accuracy and Confusion Matrix for each repeated K fold
calculateAccuracyAndConfusion method performs RepeatedKFold cross evaluation on the model to result out the best accuracy and prints in the console along with the confusion matrix. This method takes in the model, train dataset, output of train dataset and name of the model as a string so that we can do a boxplot comparison of all the algorithms.

In [13]:
def calculateAccuracyAndConfusion(model, X, y, name = ""):
    print()
    print('****************** ACCURACY *******************')
    print()

    kf = RepeatedKFold(n_splits = 3, n_repeats = 5, random_state = None) 
    results = cross_val_score(model, X, y, cv=kf)
    resultsDict[name] = results
    #Accuracy measure 
    print("Accuracy: %.3f%% (%.3f%%)" % (results.mean() * 100.0, results.std() * 100.0))

    print()
    print('****************** CONFUSION MATRIX FOR EACH REPEATED K FOLD *******************')
    print()

    for train_index, test_index in kf.split(X):
        
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model.fit(X_train, y_train)
        print(confusion_matrix(y_test, model.predict(X_test)))

##### Plot of comparison of performance beyween various algorithms

In [14]:
def showPerformancePlot():
    fig = plt.figure()
    fig.suptitle('Algorithms Comparison')
    ax = fig.add_subplot(111)
    plt.boxplot(list(resultsDict.values()))
    ax.set_xticklabels(list(resultsDict.keys()))
    plt.savefig('performance.png')
    plt.show()

### Step 3: ML algorithms covered in class

### Logistic Regression

Logistic regression is used when a classification has to be performed on the dependent variables. For example, to predict if an email is spam or not spam and if a child is happy or sad. The method logistic() takes Train dataset,  output of train dataset, test datasetand boolean value for preprocessing. In the implementation, a range of values from -2 to 3 is stored in the variable 'n' and variable 'r' uses a pow function on the this range for values of 'n' to give us a range of C values for predicting the accuracy for the best hyperparameter value. Using Repeated K Fold function, maximum accuracy is calculated and its corresponding C value is estimated. RobustScalar function is used for data preprocessing.

##### Parameters: 
- C: It is the inverse of regularization strength i.e. lambda. Higher the value of C, more will be the complexity and vice versa. In other words, smaller value of C means higher level of smoothening and greater values of C means lesser smoothening.

##### Drawbacks : 
- Continuous outcome cannot be predicted from logistic regression.
- It is mandatory that all the data points should be independent of other data points. If they are dependent on each other, then the model will tend to overweight.

In [28]:
def logistic(X, y, test, applyScalar = False):
    print()
    print('************** KCROSS VALIDATION AND LOGISTIC REGRESSION ***************')
    print()
    print('+++++++ FIND VALUE OF C +++++++')
    print()

    n = np.arange(-2,3)
    r = pow(float(10),n)
    bestCValue = 1.0
    maxAccuracy = 0

    kf = RepeatedKFold(n_splits = 3, n_repeats = 5, random_state = None) 
    for c in r:
        maxScoreValues = []
    
        for train_index, test_index in kf.split(X):
            X_train, X_test = X[train_index], X[test_index] 
            y_train, y_test = y[train_index], y[test_index]

            if applyScalar:
                scaler = RobustScaler().fit(X_train) #Scale cause of different variations in data set
                X_train = scaler.transform(X_train)
                X_test = scaler.transform(X_test)

            lr = LogisticRegression(C = c).fit(X_train, y_train)
            # print('\n'"Training Accuracy of L1 LogRess with C=%f:%f" % (c, lr_l1.score(X_train,y_train)))
            # print('\n'"Test Accuracy of L1 LogRegss with C=%f: %f" % (c, lr_l1.score(X_test,y_test)))
            maxScoreValues.append(lr.score(X_test, y_test))

        average = sum(maxScoreValues)/len(maxScoreValues)
        if average > maxAccuracy:
            maxAccuracy = average
            bestCValue = c

    print()
    print('+++++++ LOGISTIC +++++++')
    print()

    lr = LogisticRegression(C = bestCValue)
    
    if applyScalar:
        X = RobustScaler().fit(X).transform(X)
    
    lr.fit(X, y)
    calculateAccuracyAndConfusion(lr, X, y, 'Logistic Regression')
    probability = lr.predict(test)
    
    return probability


### Multilayer SVM

Multilayer SVM is a supervised machine learning algorithm which can be used for classification. In the implementation, the datasets along with model and parameters are passed to the bestParamCalculator function which returns the  best model and calculates the best accuracy.

##### Parameters : 
- C: It is the inverse of regularization strength. Complexity increases with higher value of C and vice versa.
- gammas: Gamma is a parameter for non linear hyper planes. Higher values of gamma tries to exactly fit the training data set.
- kernel : It selects the type of hyperplane for seperating the data.
- degrees: This parameter is used when the kernel is set to 'poly'.

##### Disadvantages:
- The probabilty estimates are not performed by multilayer SVM. Expensive five fold cross validation is used for getting the probabilty.
- It is not suitable for large datasets.
- In cases where dataset has more noise, multilayer SVM doesnt perform very well.

In [54]:
def multilayerSVM(X, y, testDataset, applyScalar = True):
    print('**************** SVM *******************')
    param_grid = {
         'C': [0.1, 1, 10, 100, 1000],
         'gamma' : [0.001, 0.01, 0.1, 1],
         'kernel' : ['linear','rbf', 'poly'],
         'degree' : [0, 1, 2, 3, 4, 5, 6]
              } 

    bestModel = bestParamCalculator(X, y, SVC(), param_grid, applyScalar)
    probability = bestModel.predict(testDataset)
    calculateAccuracyAndConfusion(bestModel, X, y, 'Multilayer SVM')
    return probability

### Naive Bayes
Naive Bayes classifier is a supervised classification model based on conditional probability which helps us to predict new instances based on the previous ones. We have implements two popular Naive Bayes algorithms, namely:
- Multinomial Naive Bayes 
- Gaussian Naive Bayes

##### Disadvantages: 
- Naive Bayes classifier makes a very strong assumption on the shape of your data distribution.
- Data scarcity: For any possible value of a feature, you need to estimate a likelihood value by a frequentist approach.
- Continuous features: It is common to use a binning procedure to make them discrete, but if you are not careful you can throw away a lot of information.

### Multinomial Naive Bayes 

The multinomial Naive Bayes classifier is used for classification with discrete features. MultinomialNB implements the naive Bayes algorithm for multinomially distributed data and normally requires integer feature counts.

##### Parameters: 
- alpha: 'alpha' is a Smoothing parameter. 'alpha' >= 0 accounts for features not present in the learning samples and prevents    zero probabilities in further computations. Setting 'alpha' = 1 is called Laplace smoothing, while 'alpha' < 1 is called Lidstone smoothing.

In [30]:
def multinomial(X, y, testDataset, applyNormalizer = False):
    print()
    print('**************** MULTINOMIAL NB *******************')
    print()

    n = np.arange(-2,3)
    r = pow(float(10),n)

    params = {
        'alpha' : r
    }

    bestModel = bestParamCalculator(X, y, MultinomialNB(), params, applyNormalizer)
    probability = bestModel.predict(testDataset)
    calculateAccuracyAndConfusion(bestModel, X, y, 'Multinomial NB')
    return probability

### MLP Classifier:

MLPClassifier relies on an underlying Neural Network to perform the task of classification. This model optimizes the log-loss function using LBFGS or stochastic gradient descent.

##### Parameters:
- hidden_layer_sizes : The ith element represents the number of neurons in the ith hidden layer.
- activation : Activation function for the hidden layer. (‘tanh’: the hyperbolic tan function, returns f(x) = tanh(x). ‘relu’: the rectified linear unit function, returns f(x) = max(0, x))
- solver : For weight optimization. (‘sgd’: refers to stochastic gradient descent. ‘adam’: refers to a stochastic gradient-based optimizer proposed by Kingma, Diederik, and Jimmy Ba)
- alpha : Regularization parameter.
- learning_rate : Learning rate schedule for weight updates. (‘constant’: constant learning rate given by ‘learning_rate_init’. ‘adaptive’: keeps the learning rate constant to ‘learning_rate_init’ as long as training loss keeps decreasing.)

##### Disadvantages:
- MLP with hidden layers have a non-convex loss function where there exists more than one local minimum. Therefore different random weight initializations can lead to different validation accuracy.
- MLP requires tuning a number of hyperparameters such as the number of hidden neurons, layers, and iterations.
- MLP is sensitive to feature scaling.

In [31]:
def mlpClassifier(X, y, testDataset, applyScalar = False):

    print()
    print('**************** MLP Classifier *******************')
    print()

    n = np.arange(-7,1)
    r = pow(float(10),n)
    params = {
    #30 because roughly these many features we have and lastly confirming with default value
    'hidden_layer_sizes': [(30,30,30), (100,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': r,
    'learning_rate': ['constant','adaptive'],
    }
    bestModel = bestParamCalculator(X, y, MLPClassifier(max_iter=100), params, applyScalar)
    probability = bestModel.predict(testDataset)
    calculateAccuracyAndConfusion(bestModel, X, y, 'MLPClassifier')
    return probability

### Step 4: ML algorithms not covered in class

### Gaussian Naive Bayes
Gaussian Naive Bayes works on the features having continuous values which are normally distributed. 

##### Parameters: 
- var_smoothing: Portion of the largest variance of all the features that is added to variances for calculation of stability.
  Default value for this parameter is (1e-09).

In [32]:
def gaussian(X, y, testDataset, applyScalar = False):
   
    print()
    print('**************** GAUSSIAN NB ****************')
    print()

    n = np.arange(-20,-5)
    r = pow(float(10),n)
    params = {
        'var_smoothing' : r
    }

    model = GaussianNB()
    bestModel = bestParamCalculator(X, y, model, params, applyScalar)
    probability = bestModel.predict(testDataset)
    calculateAccuracyAndConfusion(model, X, y)
    return probability

### Random Forest

For evaluating the best model, the train data set, output of train data, test data set, model, parameters and preprocessing information is sent to a bestParamCalculator function which we have defined that will evaluate all the above passed arguements using Repeated K Fold function for the hyperparameter value. Using the best model, probabilty is then calculated by passing the test dataset to the predict function.

##### Parameters:
- n_estimators : It defines the number of trees in the forest.
- max_features : It defines the maximum number of features considered for splitting the node.
- max_depth : It is the maximum number of levels in each decision tree.
- min_samples_split : minimum number of data points placed in a node before the node is split.

##### Disadvantages : 
- Complexity : A lot of trees is created by this model and combines thier output. For evaluating this, it requires more computational power and resources.
- Longer training period : As it generates a lot of trees, Random Forest requires more time to compute as compared to decision trees.

In [33]:
def randomForest(X, y, testDataset, applyScalar = False):
    
    print()
    print('************ RANDOM FOREST WITH RANDOMIZED SEARCH ***************')
    print()

    n_estimators = [int(x) for x in np.linspace(start = 5, stop = 200, num = 5)]
    max_features = ['auto', 'sqrt']
    max_depth = [int(x) for x in np.linspace(1, 45, num = 3)]
    min_samples_split = [5, 10]

    random_grid = { 
                'n_estimators': n_estimators, 
                'max_features': max_features, 
                'max_depth': max_depth, 
                'min_samples_split': min_samples_split
    }

    bestModel = bestParamCalculator(X, y, RandomForestClassifier(), random_grid, applyScalar)
    probability = bestModel.predict(testDataset)
    calculateAccuracyAndConfusion(bestModel, X, y, 'Random Forest')
    return probability

### K Nearest Neighbours

K-nearest neighbors is a simple non-parametric classification technique. The number of neighbors, k, controls model flexibility and adjusts the biasvariance tradeoff.

##### Parameters:
- n_neighbors : Number of neighbors to use by default for kneighbors queries.
- algorithm : Algorithm used to compute the nearest neighbors. (‘auto’ will attempt to decide the most appropriate algorithm based on the values passed to fit method. 'KDTree' used for fast generalized N-point problems) 
- weights : Used in prediction. (‘uniform’ : uniform weights. All points in each neighborhood are weighted equally. ‘distance’ : weight points by the inverse of their distance. In this case, closer neighbors of a query point will have a greater influence than neighbors which are further away.)

##### Disadvantages:
- Does not work well with large dataset.
- Does not work well with high dimensions.
- Need feature scaling.
- Sensitive to noisy data, missing values and outliers.

In [34]:
def knn(X, y, testDataset, applyScalar = False):
    
    print()
    print('************ K NEAREST NEIGHBOURS ***************')
    print()

    random_grid = { 
                'n_neighbors': range(1,10), 
                'algorithm': ['auto', 'kd_tree'], 
                'weights' : ['distance', 'uniform']
    }

    bestModel = bestParamCalculator(X, y, KNeighborsClassifier(), random_grid, applyScalar)
    
    probability = bestModel.predict(testDataset)
    calculateAccuracyAndConfusion(bestModel, X, y)
    
    return probability

### eXtreme Gradient Boosting/ XGBoost

eXtreme Gradient Boosting or XGBoost is a library of gradient boosting algorithms optimized for modern data science problems. It utilizes the listed techniques with boosting and is bundled in a library that is simple to use. Some of the major advantages of XGBoost are that its very scalable/parallelizable, quick to execute, and typically out performs other algorithms.

##### Parameters:
- n_estimators: This is the number of trees you want to build before taking the maximum voting or averages of predictions. Higher number of trees gives you better performance but makes your code slower.
- learning_rate: This determines the impact of each tree on the final outcome. Lower values are generally preferred as they make the model robust to the specific characteristics of tree.

##### Disadvantages: 
- Depending on the learning rate of XGB the hyper parameters behave in a more robust way, along with the robustness it has a disadvantage of significant increase in the computation time whereas a significant jump can be obtained by feature engineering, creating ensembles of models which behaves in a slightly better manner as compared to time, for Example GBM (Gradient Boosting model). Also XGB requires careful tuning of hyper parameters.

In [43]:
def applyXGBoostFeatureSelection(X, y, testDataset, applyScalar = False):
    print()
    print('************ XG BOOST ***************')
    print()
    xgbClassifier = xgb.XGBClassifier(n_estimators=2000,learning_rate=0.3)
    if applyScalar:
        scaler = RobustScaler().fit(X) #Scale cause of different variations in data set
        X = scaler.transform(X)    
    xgbClassifier.fit(X, y)

### Step 5: Main code for reading csv

In [35]:
trainDataset = pd.read_csv('train.csv')
testDataset = pd.read_csv('test.csv')
y = trainDataset['Category']
X = trainDataset.iloc[:, :-1].values #without category

### Maximum likelihood

In [36]:
mle(y)


************** MAXIMUM LIKELIHOOD ***************

Maximum likelihood  0    0.807569
1    0.192431
Name: Category, dtype: float64


### Step 6: Calling various methods depending on algorithm we want to test

In [ ]:
print()
print('============== FINDING PREDICTIONS AND TRAINING ================')
print()

probabilityF = randomForest(X, y, testDataset, True)
probabilityG = gaussian(X, y, testDataset, True)
probabilityM = multinomial(X, y, testDataset)
probabilityMC = mlpClassifier(X, y, testDataset, True)
probabilityK = knn(X, y, testDataset, True)
probabilityL = logistic(X, y, testDataset, True)
probabilityA = applyXGBoostFeatureSelection(X, y, testDataset, True)
probabilityMSVM = multilayerSVM(X, y, testDataset, True)

showPerformancePlot()
# generateCSV(probability, testDataset)